## Imports

In [2]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Load

In [3]:
DATANAME = "noise_filter.pcd"
#DATANAME = "appartment_cloud.ply"
pcd = o3d.io.read_point_cloud(DATANAME)

In [5]:
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

nn = 16
std_multiplier =10

filtered_pcd = pcd.remove_statistical_outlier(nn, std_multiplier)

#outliers = pcd.select_by_index(filtered_pcd[1], invert=True)
#outliers.paint_uniform_color([1, 0, 0])
filtered_pcd = filtered_pcd[0]

o3d.visualization.draw_geometries([filtered_pcd, outliers])

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [11]:
voxel_size = 0.001
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size = voxel_size)
o3d.visualization.draw_geometries([pcd_downsampled])

In [12]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())

radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn = 16), fast_normal_computation = True)

pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd_downsampled, outliers])

In [13]:
front = [0.9681980579027637, 0.24767928031788963, -0.03531989205149242]
lookat = [0.0084975577023191553, -0.2087872942101065, -0.48907046814855626]
up = [0.011867560061902271, 0.095549553792428299, 0.99535392890581653]
zoom = 0.23999999999999969

pcd = pcd_downsampled
o3d.visualization.draw_geometries([pcd],zoom=zoom, front=front, lookat=lookat, up=up)

In [14]:
pt_to_plane_dist = 0.01

plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3, num_iterations=1000)

[a,b,c,d] = plane_model

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0,0,0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],zoom=zoom, front=front, lookat=lookat, up=up)

## Multi-order RANSAC

In [29]:
max_plane_idx = 2
pt_to_plane_dist = 0.02

segment_models = {}
segments = {}
rest = pcd

for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3, num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done")

o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])


pass 0 / 2 done
pass 1 / 2 done
